In [1]:
import pandas as pd
import requests
import json
from config import config
import mysql.connector
from mysql.connector import Error
import os

In [2]:
# # url='http://127.0.0.1:2626/v1/users/predictions'
# url='http://127.0.0.1:2626/v1/users/predictions'
# header={'Content-Type':'application/json'}

# response=requests.get(url=url, headers=header)
# print(response.status_code)
# print(response.json())

In [3]:
# Store this pandas dataframe in a MySQL database
df = pd.read_csv(os.path.join(config.DATAPATH, 'test.csv'), encoding="utf-8-sig")
df = df.iloc[:300]
df = df.to_json(orient='records')

url='http://127.0.0.1:2626/v1/batch/predictions'
header={'Content-Type':'application/json', 'Accept': 'application/json'}

response=requests.post(url=url, data=json.dumps(df), headers=header)
print(response.status_code)

200


In [4]:
idx=[]
label=[]
score=[]
tweet=[]
for num,v in enumerate(response.json()['outputs']):
    idx.append(num)
    label.append(v['predictedLabel'])
    score.append(v['probabilityScore'])
    tweet.append(v['tweet'])
    
df_pred = pd.DataFrame(list(zip(idx, tweet, label, score)), columns=['id','tweet',
                                                                     'predictedLabel',
                                                                     'predictedProbability'])

In [5]:
df_pred.head(60)

,id,tweet,predictedLabel,predictedProbability
0,0,#studiolife #aislife #requires #passion #dedic...,0,0.004470
1,1,@user #white #supremacists want everyone to s...,0,0.023741
2,2,safe ways to heal your #acne!! #altwaystohe...,0,0.000598
3,3,is the hp and the cursed child book up for res...,0,0.003823
4,4,"3rd #bihday to my amazing, hilarious #nephew...",0,0.000008
5,5,choose to be :) #momtips,0,0.000592
6,6,something inside me dies ð¦ð¿â¨ eyes nes...,0,0.001464
7,7,#finished#tattoo#inked#ink#loveitâ¤ï¸ #â¤ï¸...,0,0.006116
8,8,@user @user @user i will never understand why...,0,0.050262
9,9,#delicious #food #lovelife #capetown mannaep...,0,0.005353


In [6]:
# The model.bin version 1 is not producing good results, we need to improve the model
df_pred.iloc[245]['tweet']

' @user i can\'t believe how much i used to care what people thought of me now i\'m just like "lol fuck u, fuck you and  wtf fuck youâ\x80¦'

In [7]:
df_pred.iloc[245]

id                                                                    245
tweet                    @user i can't believe how much i used to care...
predictedLabel                                                          0
predictedProbability                                             0.064127
Name: 245, dtype: object

In [8]:
df_pred.dtypes

id                        int64
tweet                    object
predictedLabel            int64
predictedProbability    float64
dtype: object

In [9]:
class Stroredata():
    def __init__(self, password, host, database, user):
        
        self.password = password
        self.host = host
        self.database = database
        self.user = user

    def createDB(self):

        try:
            connection = mysql.connector.connect(
                host=self.host, 
                user=self.user, 
                password=self.password, 
                database=self.database
            )

            cursor = connection.cursor()
            cursor.execute(
                """CREATE DATABASE batchtweetdb"""
            )
            print("Database created!")
        except Error as e:
            print(f"Error while connecting to MySQL{e}")
            print('\nDatabases:')
            cursor.execute("SHOW DATABASES")
            for x in cursor:
                print(x)

    def insertData(self, database='batchtweetdb'):
        try:
            connection = mysql.connector.connect(
                    host=self.host, 
                    user=self.user, 
                    password=self.password, 
                    database=database
                )
            if connection.is_connected():
                cursor = connection.cursor()
                # cursor.execute('DROP TABLE IF EXISTS tweetpreds;')
                print("Creating table"+"..."*2)
                table="""
                CREATE TABLE IF NOT EXISTS tweetpreds(
                    id INT NOT NULL PRIMARY KEY,
                    tweet TEXT,
                    predictedLabel INT NOT NULL,
                    predictedProbability DECIMAL(8,8) NOT NULL

                )
                """
                cursor.execute(table)
                print("Tweet with predicitons table created"+"..."*2)
                for i, row in df_pred.iterrows():
                    sql_query = """
                    INSERT INTO batchtweetdb.tweetpreds
                    VALUES
                    (%s,%s,%s,%s);
                    """
                    cursor.execute(sql_query, tuple(row))
                    connection.commit()
        except Error as e:
            print(e)
        
        cursor.close()
        connection.close()
        print('Record inserted!')

    def displayData(self, database='batchtweetdb'):
        try:
            connection = mysql.connector.connect(
                    host=self.host, 
                    user=self.user, 
                    password=self.password, 
                    database=database
                )
            if connection.is_connected():
                cursor = connection.cursor()
                cursor.execute(
                    """
                    SELECT * FROM batchtweetdb.tweetpreds;
                    """
                )
        except Error as e:
            print(e)
        for entry in cursor.fetchall():
            print(entry)

In [10]:
b=Stroredata(config.PASSWORD, config.HOST, '', config.USER)
b.insertData()
b.displayData()

Creating table......
Tweet with predicitons table created......
1062 (23000): Duplicate entry '0' for key 'tweetpreds.PRIMARY'
Record inserted!
(0, '#studiolife #aislife #requires #passion #dedication #willpower   to find #newmaterialsâ\x80¦ ', 0, Decimal('0.00446951'))
(1, ' @user #white #supremacists want everyone to see the new â\x80\x98  #birdsâ\x80\x99 #movie â\x80\x94 and hereâ\x80\x99s why  ', 0, Decimal('0.02374113'))
(2, 'safe ways to heal your #acne!!    #altwaystoheal #healthy   #healing!! ', 0, Decimal('0.00059754'))
(3, 'is the hp and the cursed child book up for reservations already? if yes, where? if no, when? ð\x9f\x98\x8dð\x9f\x98\x8dð\x9f\x98\x8d   #harrypotter #pottermore #favorite', 0, Decimal('0.00382314'))
(4, '  3rd #bihday to my amazing, hilarious #nephew eli ahmir! uncle dave loves you and missesâ\x80¦ ', 0, Decimal('0.00000839'))
(5, 'choose to be   :) #momtips ', 0, Decimal('0.00059185'))
(6, 'something inside me dies ð\x9f\x92¦ð\x9f\x92¿â\x9c¨  eyes ness #sm